In [3]:
import sys
sys.path.append('..')

In [4]:
from importlib import reload
from os.path import exists

import networkx as nx
import numpy as np
import pandas as pd
import lib.obo
import lib.classification.elastic
import lib.classification.validation
import lib.utils
import data
import pymonad

reload(lib.classification.validation)
reload(lib.classification.elastic)
reload(lib.obo)
reload(lib.utils)
reload(data)

from lib.classification.elastic import annotate_index, collapse_matches, build_synonyms_graph, search_item
from lib.utils import map_series, expand_dataframe, expand, flatten
from lib.classification.validation import correct_ratio, incorrect_ratio, perfect_match
from geo_annotation.import_ontology_es import import_ontology
from elasticsearch import Elasticsearch
from lib.obo import read_ontology, Ontology
pd.set_option('display.width', 512)

In [5]:
import json

In [44]:
def read_export(file_name, read_cols):
    res = []
    lines_read = 0
    with open(file_name) as f:
        for l in f:
            lines_read += 1
            if lines_read % 10000 == 0:
                print("Read #{} lines".format(lines_read))
            l = l.rstrip("\n")
            data = json.loads(l)
            yield [data.get(c, np.nan) for c in read_cols]     

In [14]:
uc_file = '/data/rawdata/snapshots/mongo/scraper_test_dev.samples.age.uc.json'
lc_file = '/data/rawdata/snapshots/mongo/scraper_test_dev.samples.age.lc.json'

In [20]:
from itertools import chain

In [45]:
columns =  ['accession', 'characteristics', 'series', 'platform', 'organism']
res = pd.DataFrame.from_records(data=chain(read_export(uc_file, columns), read_export(lc_file, columns)), columns=columns)

Read #10000 lines
Read #20000 lines
Read #30000 lines
Read #40000 lines
Read #50000 lines
Read #60000 lines
Read #70000 lines
Read #80000 lines
Read #90000 lines
Read #100000 lines
Read #110000 lines
Read #120000 lines
Read #130000 lines
Read #140000 lines
Read #150000 lines
Read #160000 lines
Read #10000 lines
Read #20000 lines
Read #30000 lines
Read #40000 lines
Read #50000 lines
Read #60000 lines
Read #70000 lines
Read #80000 lines
Read #90000 lines
Read #100000 lines
Read #110000 lines
Read #120000 lines
Read #130000 lines
Read #140000 lines
Read #150000 lines
Read #160000 lines
Read #170000 lines
Read #180000 lines
Read #190000 lines
Read #200000 lines
Read #210000 lines
Read #220000 lines
Read #230000 lines
Read #240000 lines
Read #250000 lines
Read #260000 lines


In [46]:
res.head()

,accession,characteristics,series,platform,organism
0,E-MTAB-452_10,"{'Organism': 'Mus musculus', 'Age': ' ', 'Cell...",[E-MTAB-452],A-AFFY-23,Mus musculus
1,E-MTAB-3152_0,"{'initial time point': ' ', 'Age': ' ', 'tis...",[E-MTAB-3152],A-MTAB-535,Homo sapiens
2,E-MTAB-3152_2,"{'initial time point': ' ', 'Age': ' ', 'tis...",[E-MTAB-3152],A-MTAB-535,Homo sapiens
3,E-MTAB-3152_1,"{'initial time point': ' ', 'Age': ' ', 'tis...",[E-MTAB-3152],A-MTAB-535,Homo sapiens
4,E-MTAB-3152_3,"{'initial time point': ' ', 'Age': ' ', 'tis...",[E-MTAB-3152],A-MTAB-535,Homo sapiens


In [52]:
def get_age(c):
    age = c.get('age') or c.get('Age', '')
    if isinstance(age, list):
        age = age[0]
    age = age.strip(' ')
    return age if age else np.nan

def get_organism(o):
    return o if not isinstance(o, list) else o[0]


In [53]:
res['age'] = res['characteristics'].map(get_age)
res['organism'] = res['organism'].map(get_organism)

In [48]:
res.head()

,accession,characteristics,series,platform,organism,age
0,E-MTAB-452_10,"{'Organism': 'Mus musculus', 'Age': ' ', 'Cell...",[E-MTAB-452],A-AFFY-23,Mus musculus,NaN
1,E-MTAB-3152_0,"{'initial time point': ' ', 'Age': ' ', 'tis...",[E-MTAB-3152],A-MTAB-535,Homo sapiens,NaN
2,E-MTAB-3152_2,"{'initial time point': ' ', 'Age': ' ', 'tis...",[E-MTAB-3152],A-MTAB-535,Homo sapiens,NaN
3,E-MTAB-3152_1,"{'initial time point': ' ', 'Age': ' ', 'tis...",[E-MTAB-3152],A-MTAB-535,Homo sapiens,NaN
4,E-MTAB-3152_3,"{'initial time point': ' ', 'Age': ' ', 'tis...",[E-MTAB-3152],A-MTAB-535,Homo sapiens,NaN


In [49]:
res.dropna(subset=['age']).shape

(405833, 6)

In [54]:
res.to_pickle('../data/geo-annotation/samples.age.uncleaned.v2.pickle')

In [55]:
res.dropna(subset=['age']).query("organism == 'Homo sapiens'").shape

(176150, 6)

In [56]:
res = res.dropna(subset=['age']).query("organism == 'Homo sapiens'")

In [57]:
res.head()

,accession,characteristics,series,platform,organism,age
22831,E-MTAB-62_4017,"{'4 meta-groups': 'disease', 'DataSource': 'GS...",[E-MTAB-62],A-AFFY-33,Homo sapiens,"""0,91"""
22869,E-MTAB-872_0,"{'Organism': 'Homo sapiens', 'CellType': 'embr...",[E-MTAB-872],A-GEOD-10558,Homo sapiens,0
22870,E-MTAB-872_1,"{'Organism': 'Homo sapiens', 'CellType': 'embr...",[E-MTAB-872],A-GEOD-10558,Homo sapiens,0
22871,E-MTAB-872_2,"{'Organism': 'Homo sapiens', 'CellType': 'embr...",[E-MTAB-872],A-GEOD-10558,Homo sapiens,0
22890,E-MTAB-487_0,"{'Organism': 'Homo sapiens', 'OrganismPart': '...",[E-MTAB-487],A-GEOD-8490,Homo sapiens,0


In [59]:
res['age'] = res['age'].astype(str)

In [60]:
res['age'] = res['age'].str.replace(',', '.')

In [61]:
import fastnumbers as fn

In [62]:
res['isfloat'] = res['age'].map(fn.isfloat)

In [63]:
res['isfloat'].value_counts()

True     139273
False     36877
dtype: int64

In [65]:
res[res['isfloat'] == False].age.value_counts()

na                           1607
n/a                           846
NA                            789
adult                         728
not available                 665
--                            658
>=60 years                    445
not collected                 376
50-59 years                   298
newborn                       256
12 yrs                        247
unknown                       245
day 4 embryo                  240
not applicable                233
60-69 years                   233
21 pcw                        206
>= 60 years                   200
12 pcw                        186
<60 years                     185
20-45                         184
13 pcw                        174
43 years                      172
40-49 years                   171
56 years                      169
young                         165
46 years                      165
49 years                      164
4 pmonth                      160
none                          160
55 years      

In [66]:
res[res['isfloat']].to_pickle('../data/geo-annotation/samples.age.cleaned.v1.pickle')